Final Project - Churn Part B

In [1]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
import pickle

client = MongoClient("mongodb://localhost:27017/")
db = client['BDA']
collection = db['Customers']

with open(r'C:\Users\emmag\Python\Final Project\churn_knn_model.sav', 'rb') as f:
    model = pickle.load(f)

In [3]:
def get_all_fields(collection):
    all_fields = set()
    for doc in collection.find():
        all_fields.update(doc.keys())
    all_fields.discard('_id')
    return all_fields

all_fields = get_all_fields(collection)

query = {field: {'$exists': True} for field in all_fields}

documents = list(collection.find(query, {'_id': False}))

df = pd.DataFrame(documents)
services_df = pd.json_normalize(df['Services'])
df = df.drop(columns=['Services'])
df = pd.concat([df, services_df], axis=1)
df = df.rename(columns=str.lower)

df

,customerid,gender,seniorcitizen,partner,dependents,tenure,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies
0,id7044,Male,0,Yes,No,15.208851,Month-to-month,[No],Credit card (automatic),45.682782,651.55,Yes,No,DSL,No internet service,Yes,No internet service,Yes,No,Yes
1,id7045,Male,0,No,Yes,29.779905,Month-to-month,[Yes],Credit card (automatic),40.301502,1359.7,No,Yes,DSL,No internet service,Yes,Yes,No internet service,Yes,Yes
2,id7046,Female,0,Yes,Yes,34.546357,Month-to-month,[Yes],Mailed check,59.778062,1752.55,Yes,No,DSL,Yes,No,Yes,No,No internet service,No internet service
3,id7047,Male,0,No,No,37.264434,Month-to-month,[Yes],Electronic check,58.392744,541.9,Yes,Yes,DSL,Yes,No,Yes,No internet service,No,Yes
4,id7049,Female,1,No,No,31.188870,Two year,[No],Bank transfer (automatic),87.322896,845.6,Yes,Yes,Fiber optic,No,Yes,Yes,No internet service,No internet service,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23418,id32038,Male,0,No,No,10.304650,Month-to-month,[Yes],Credit card (automatic),91.817322,867.3,Yes,Yes,Fiber optic,No internet service,Yes,Yes,Yes,Yes,No internet service
23419,id32039,Male,0,No,Yes,15.826004,Month-to-month,[No],Mailed check,71.692238,7962.2,Yes,Yes,DSL,Yes,No,No,No,No,No internet service
23420,id32040,Male,0,No,Yes,33.852572,One year,[Yes],Mailed check,86.958880,374.8,Yes,Yes,Fiber optic,No internet service,No internet service,No,Yes,Yes,No
23421,id32041,Male,0,Yes,Yes,33.154407,Month-to-month,[Yes],Mailed check,76.120814,238.15,Yes,Yes,Fiber optic,No,Yes,No,No,No,No internet service


In [4]:
label_encoder_contract = preprocessing.LabelEncoder()
label_encoder_internetservice = preprocessing.LabelEncoder()
label_encoder_paymentmethod = preprocessing.LabelEncoder()

df['contract_code'] = label_encoder_contract.fit_transform(df['contract'])
contract_mapping = {index: label for index, label in enumerate(label_encoder_contract.classes_)}

df['internetservice_code'] = label_encoder_internetservice.fit_transform(df['internetservice'])
internetservice_mapping = {index: label for index, label in enumerate(label_encoder_internetservice.classes_)}

df['paymentmethod_code'] = label_encoder_paymentmethod.fit_transform(df['paymentmethod'])
paymentmethod_mapping = {index: label for index, label in enumerate(label_encoder_paymentmethod.classes_)}



df['totalcharges'] = pd.to_numeric(df['totalcharges'], errors='coerce')

def convert_yes_no_to_int(df, column_name):
    df.loc[df[column_name] == 'No', column_name] = 0
    df.loc[df[column_name] == 'Yes', column_name] = 1

    df.loc[~df[column_name].isin([0, 1]), column_name] = 0  
    df[column_name] = df[column_name].astype('int64')

columns_to_convert = [
    'partner', 'dependents', 'phoneservice', 'multiplelines',
    'onlinesecurity','onlinebackup', 'deviceprotection', 'techsupport',
    'streamingtv', 'streamingmovies', 'paperlessbilling'
]

for column in columns_to_convert:
    convert_yes_no_to_int(df, column)

df.loc[df.gender == 'Male', 'gender'] = 0
df.loc[df.gender == 'Female', 'gender'] = 1
df = df.astype({"gender": 'int64'})


df = df.drop(columns=['contract','paymentmethod','internetservice'])

df['churn'] = 0
df['churn'] = np.random.uniform(0, 1, df.shape[0])
df['churn'] = df['churn'].round().astype(int)

df.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,paperlessbilling,monthlycharges,totalcharges,phoneservice,...,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract_code,internetservice_code,paymentmethod_code,churn
0,id7044,0,0,1,0,15.208851,0,45.682782,651.55,1,...,0,1,0,1,0,1,0,0,1,0
1,id7045,0,0,0,1,29.779905,0,40.301502,1359.70,0,...,0,1,1,0,1,1,0,0,1,0
2,id7046,1,0,1,1,34.546357,0,59.778062,1752.55,1,...,1,0,1,0,0,0,0,0,3,1
3,id7047,0,0,0,0,37.264434,0,58.392744,541.90,1,...,1,0,1,0,0,1,0,0,2,1
4,id7049,1,1,0,0,31.188870,0,87.322896,845.60,1,...,0,1,1,0,0,0,2,1,0,1


In [5]:
expected_features = ['gender', 'seniorcitizen', 'partner', 'dependents',
                     'tenure', 'phoneservice', 'multiplelines', 'internetservice_code', 'onlinesecurity',
                     'onlinebackup', 'deviceprotection', 'techsupport', 'streamingtv', 'streamingmovies',
                     'contract_code', 'paperlessbilling', 'paymentmethod_code', 'monthlycharges','totalcharges']

for feature in expected_features:
    if feature not in df.columns:
        print(f"Feature {feature} is missing from the data.")
        df[feature] = 0

df_filtered = df[expected_features]
df_filtered.head()

,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice_code,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract_code,paperlessbilling,paymentmethod_code,monthlycharges,totalcharges
0,0,0,1,0,15.208851,1,0,0,0,1,0,1,0,1,0,0,1,45.682782,651.55
1,0,0,0,1,29.779905,0,1,0,0,1,1,0,1,1,0,0,1,40.301502,1359.70
2,1,0,1,1,34.546357,1,0,0,1,0,1,0,0,0,0,0,3,59.778062,1752.55
3,0,0,0,0,37.264434,1,1,0,1,0,1,0,0,1,0,0,2,58.392744,541.90
4,1,1,0,0,31.188870,1,1,1,0,1,1,0,0,0,2,0,0,87.322896,845.60


In [6]:
imputer = SimpleImputer(strategy='mean')
df_imputed = pd.DataFrame(imputer.fit_transform(df_filtered), columns=df_filtered.columns)
df_imputed.head()

,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice_code,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract_code,paperlessbilling,paymentmethod_code,monthlycharges,totalcharges
0,0.0,0.0,1.0,0.0,15.208851,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,45.682782,651.55
1,0.0,0.0,0.0,1.0,29.779905,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,40.301502,1359.70
2,1.0,0.0,1.0,1.0,34.546357,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,59.778062,1752.55
3,0.0,0.0,0.0,0.0,37.264434,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2.0,58.392744,541.90
4,1.0,1.0,0.0,0.0,31.188870,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,87.322896,845.60


In [7]:
df_pred_array = df_imputed.to_numpy()

print("Data to be predicted:")
print(df_imputed.head())

Data to be predicted:
   gender  seniorcitizen  partner  dependents     tenure  phoneservice  \
0     0.0            0.0      1.0         0.0  15.208851           1.0   
1     0.0            0.0      0.0         1.0  29.779905           0.0   
2     1.0            0.0      1.0         1.0  34.546357           1.0   
3     0.0            0.0      0.0         0.0  37.264434           1.0   
4     1.0            1.0      0.0         0.0  31.188870           1.0   

   multiplelines  internetservice_code  onlinesecurity  onlinebackup  \
0            0.0                   0.0             0.0           1.0   
1            1.0                   0.0             0.0           1.0   
2            0.0                   0.0             1.0           0.0   
3            1.0                   0.0             1.0           0.0   
4            1.0                   1.0             0.0           1.0   

   deviceprotection  techsupport  streamingtv  streamingmovies  contract_code  \
0               0.0

In [8]:
predictions = model.predict(df_pred_array)

churn_knn_output1 = pd.DataFrame({
    'customerid': df['customerid'],
    'actually_churned': df['churn'],
    'churned_predicted_by_model': predictions
})


churn_knn_output1

,customerid,actually_churned,churned_predicted_by_model
0,id7044,0,0.0
1,id7045,0,0.0
2,id7046,1,0.0
3,id7047,1,0.0
4,id7049,1,0.0
...,...,...,...
23418,id32038,0,0.0
23419,id32039,1,0.0
23420,id32040,0,0.0
23421,id32041,0,0.0


In [9]:
total_predicted_churn = churn_knn_output1['churned_predicted_by_model'].sum()
total_predicted_churn

0.0

In [14]:
df = df.merge(churn_knn_output1[['customerid', 'churned_predicted_by_model']], on='customerid', how='left')
df = df.drop(columns=['churn'])
df.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,paperlessbilling,monthlycharges,totalcharges,phoneservice,...,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract_code,internetservice_code,paymentmethod_code,churned_predicted_by_model
0,id7044,0,0,1,0,15.208851,0,45.682782,651.55,1,...,0,1,0,1,0,1,0,0,1,0.0
1,id7045,0,0,0,1,29.779905,0,40.301502,1359.70,0,...,0,1,1,0,1,1,0,0,1,0.0
2,id7046,1,0,1,1,34.546357,0,59.778062,1752.55,1,...,1,0,1,0,0,0,0,0,3,0.0
3,id7047,0,0,0,0,37.264434,0,58.392744,541.90,1,...,1,0,1,0,0,1,0,0,2,0.0
4,id7049,1,1,0,0,31.188870,0,87.322896,845.60,1,...,0,1,1,0,0,0,2,1,0,0.0


In [12]:
df.to_csv('churn_pred_partB.csv', index=False)

print("The merged DataFrame has been saved to 'churn_pred_partB.csv'.")

The merged DataFrame has been saved to 'churn_pred_partB.csv'.
